<a href="https://colab.research.google.com/github/Rodzeymer/Imersao_Python/blob/main/Imers%C3%A3o_Python_Aula_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nesta Aula 03, vamos trabalhar puramente com Python e para isso precisamos estruturar nossas ideias e os códigos, pois eles devem seguir uma cadeia lógica de instruções, de forma que quando executarmos todas as intruções a direção da execução seja linear e direta, ou seja, começa em um ponto e termina em outro sem fazer curvas e loops, e aqui vamos fazer de cima para baixo, ou como é conhecido Drop-down.

Para facilitar a leitura irei comentar dentro dos blocos de código usando a cerquilha, conhecido como HashTag.

In [81]:
# Nesse primeiro bloco que deixei em separado vão as importações de bibliotecas,
#sempre que uma biblioteca nova precisar ser adicionada ela será adicionada aqui.

import pandas as pd
import plotly.express as px

In [82]:
# Começando o tratamento dos dados precisamos primeiro carregar o arquivo a ser
# trabalhado ou que servirá como base de dados para as análises, então adicionamos
# ele no Notebook e carregamos ele pelo Pandas, fazendo sua leitura.
# Podemos atribuir cada Planilha a uma variável, facilitando sua manipulação

caminhoArquivo = "/content/Planilha auxiliar - Imersão Python - Tabela de ações.xlsx"

df_principal = pd.read_excel(caminhoArquivo, sheet_name="Principal")
df_total_acoes = pd.read_excel(caminhoArquivo, sheet_name="Total_de_acoes")
df_ticker = pd.read_excel(caminhoArquivo, sheet_name="Ticker")
df_ChatGPT = pd.read_excel(caminhoArquivo, sheet_name="Chatgpt")

In [ ]:
# Aqui começamos a montar nossa tabela principal, podemos também criar uma nova planilha,
# mas como já temos uma com vários dados que precisamos vamos sobreescrevê-la
# com os dados que nos interessa

df_principal = df_principal[["Ativo", "Data", "Último (R$)", "Var. Dia (%)"]].copy()


In [ ]:
# À medida que iremos adicionando novas colunas ao nosso dataframe, que é um conjunto de vetores de dados,
# vamos renomeando os títulos que não estiverem atendendo a padrões a a convenções
# de boas práticas de programação, evitando que os códigos quebrem devido a espaços vazios
# e a acentos e caracteres especiais, que dependendo da linguagem pode não suportá-los.

df_principal = df_principal.rename(columns={"Último (R$)":"Valor_Final", "Var. Dia (%)":"Var_Dia_Pct"}).copy()

In [ ]:
# Aqui começamos a criar novas colunas, como por exemplo a variação em percentual, já que a coluna "Var_Dia_Pct"
# apresenta a variação em decimal, a coluna "Var_Pct" agora mostra o valor em percentual, agora podemos
# calcular o valor inicial, colocando os valores devidos na coluna "Valor_Inicial

df_principal['Var_Pct'] = df_principal["Var_Dia_Pct"] / 100
df_principal["Valor_Inicial"] = df_principal["Valor_Final"] / (df_principal['Var_Pct'] + 1)

In [ ]:
# Além de criar novas colunas, podemos pegar colunas de outras tabelas, lembrando que temos que
# carregalas através do Pandas para termos acesso a elas.

# Aqui faremos o Merge, ou junção, do dataframe "df_total_acoes", é importante que as tabelas
# tenham uma coluna em comum, a chamada chave estrangeira, foreing key (FK), para que os dados
# fiquem organizados. Veja que indicamos como chave estrangeira no "df_principal" a coluna "Ativo",
# que corresponde à coluna "Código" do "df_total_acoes", e o sentido de inclusão foi da direita (right)
# para a esquerda, logo a "df_principal" serve de base e ganha as colunas do "df_total_acoes"

df_principal = df_principal.merge(df_total_acoes, left_on="Ativo", right_on="Código", how="left")


In [ ]:
# Agora observe que o df_principal possui duas colunas de iguais, as colunas Ativo
# e Código, precisamos fazer a exclusão de uma dessas, como a Código está à direita da tabela,
# se misturando aos dados numéricos faremos o drop dela, a função Drop descarta a coluna indicada,
# como se vê abaixo.

df_principal = df_principal.drop(columns=["Código"])

In [ ]:
# Temos agora a coluna de Qtdr Teórica, que indica uma quantidade teórica de ações
# disponíveis na data da aquisição dos dados, com o Valor_Final e Valor_Inicial
# podemos calcular a sua variação em reais e calcular o valor da variação de cada ação

df_principal["Variacao_RS"] = (df_principal["Valor_Final"] - df_principal["Valor_Inicial"]) * df_principal['Qtde. Teórica']

In [126]:
# Como nos dados do tipo float estamos tratando de dados que só nos importam até
# a segunda casa decimal vamos formatar esses números para apresentar apenas
# essas duas casas decimais, alterando uma opção global do Pandas.

pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Vejamos que a coluna Qtde_Teórica estaá sendo tratada como do tipo float, como
# não existe fração de ação vamos trocar esse tipo para inteiro, descartando assim
# as casas decimais da coluna, sem alterar os seus valores.

df_principal["Qtde. Teórica"] = df_principal["Qtde. Teórica"].astype(int)

In [ ]:
# Precisamos renomear o título dessa coluna também

df_principal = df_principal.rename(columns={"Qtde. Teórica":"Qtde_Teorica"}).copy()

In [ ]:
# Agora vamos começar nossas análises, primeiramente vamos separar as ações
# de quem subiu, ficou estável e desceu, usando uma fórmula simples.

# O comando .apply permite executar uma função em todos os elementos de uma
# coluna especificada, nesse caso criamos a coluna Resultado, atribuindo a ela a
# coluna Variacao_RS e aplicando a essa coluna a seguinte função.
# Chamamos lambda para informar que os dados da coluna Variacao_RS serão
# submetidos à seguinte # fórmula, o resultado será "Subiu", se X (a Variacao_RS no caso)
# for maior que 0 # ou será "Desceu" se X for menor que 0,
# se a Variacao_RS não subiu (X>0) e nem desceu (X<0) então o resultado será "Estável"

df_principal["Resultado"] = df_principal["Variacao_RS"].apply(lambda x: "Subiu" if x >0 else("Desceu" if x <0 else "Estável"))

In [ ]:
# Neste passo iremos fazer mais um merge, agora com o df_ticker, pegando as colunas
# Ativo do df_principal e Ticker do df_ticker como chaves em comum, e puxando as
# colunas de df_ticker para df_principal, e já incluímos o drop da coluna Ticker.

# vemos que a cada merge fazemos um drop, pois, em geral, as duas colunas de
# correspondência possuem os mesmos dados, por isso elas são usadas como critério de ordenação

df_principal = df_principal.merge(df_ticker, left_on="Ativo", right_on="Ticker", how="left")
df_principal = df_principal.drop(columns=["Ticker"])


In [ ]:
# Mais um merge e mais um drop, agora pegamos as colunas do df_Chatgpt, pareando as
# colunas Nome e Nome da empresa, do df_principal e df_Chatgpt, respectivamente,
# fazendo o drop da coluna Nome da empresa ao final.

df_principal = df_principal.merge(df_ChatGPT, left_on="Nome", right_on="Nome da empresa", how="left")
df_principal = df_principal.drop(columns=["Nome da empresa"])

In [ ]:
# Seguindo as boas boas práticas de programação faremos a renomeação do título da
# coluna Idade (anos) para Idade apenas.

df_principal = df_principal.rename(columns={"Idade (anos)":"Idade"}).copy()

In [ ]:
# Agora mais uma análise interessante, vamos categorizar as empresas pela sua idade
# veja que usamos mais uma vez a função lamdba, para aplicar a nossa fórmula em
# toda a coluna Idade, guardando os resultados em Cat_Idade e com isso podemos
# verificar quantas empresas estão em cada catergoria e quanto de variação cada
# categoria agregou nesse dia.

df_principal["Cat_Idade"] = df_principal["Idade"].apply(lambda x: 'Mais de 100' if x>100 else ('Menos de 50' if x<50 else 'Entre 50 e 100'))


In [97]:
# E nessa parte vamos verificar qual foi a ação que mais subiu e qual mais desceu
# além de ver a média da variação, e podemos ainda ver qual a média de subida e
# a média da descida

maior = df_principal['Variacao_RS'].max()
menor = df_principal['Variacao_RS'].min()
media = df_principal ['Variacao_RS'].mean()
media_subiu = df_principal[df_principal["Resultado"] == "Subiu"]["Variacao_RS"].mean()
media_desceu = df_principal[df_principal["Resultado"] == "Desceu"]["Variacao_RS"].mean()

# Imprimindo Resultados
print(f"Maior\t R$ {maior:,.2f}")
print(f"Menor\t R$ {menor:,.2f}")
print(f"Média\t R$ {media:,.2f}")
print(f"Média de quem subiu\t R$ {media_subiu:,.2f}")
print(f"Média de quem desceu\t R$ {media_desceu:,.2f}")


Maior	 R$ 4,762,926,995.25
Menor	 R$ -1,807,432,634.46
Média	 R$ 218,008,898.33
Média de quem subiu	 R$ 538,720,638.36
Média de quem desceu	 R$ -186,265,310.46


In [ ]:
# Podemos analisar as ações que subiram e colocar elas em um dataframe
# separado, como fizemos nesse df_principal_subiu

df_principal_subiu = df_principal[df_principal["Resultado"] == "Subiu"]

In [ ]:
# E por fim podemos plotar gráficos das análises, nessa figura 1 colocamos em
# barras a variação por resultado, ou seja o total de variação positiva e o total
# de variação negativa.

figura_1 = px.bar(df_analise_saldo, x='Resultado', y='Variacao_RS', text='Variacao_RS',
                  title="Variação Reais por Resultado", color="Resultado", height=500, width=800)

figura_1.update_yaxes(tickprefix="R$ ")
figura_1.update_traces(texttemplate="R$ %{text:,.2f}")
figura_1.add_hline(y=0, line_color='black', line_dash="dot")

figura_1.show()

In [ ]:
# Podemos fazer uma análise por segmentos, verificando a variação final de cada
# segmento e sua participação na variação geral das ações.
figura_2 = px.pie(df_analise_segmento, values='Variacao_RS', names='Segmento', title='Variação Reais por Segmento', height=500, width=800)
figura_2.update_traces(textposition='inside', textinfo='percent+label', hovertemplate="Variação: R$ %{value:,.2f}")

figura_2.show()


In [ ]:
# E analisar finalmente a variação final por categoria de idade, podendo extrair
# informações interessantes, já que se que as empresas jovens subiram mais que as
# centenárias, mas que as empresas maduras subiram bem mais, podendo essa variação
# exarcebada ser devido a uma verdadeira valorização das suas ações o pela quantidade
# de empresas maduras que constam nesse banco de dados.

df_analise_idade = df_principal.groupby("Cat_Idade").agg({"Variacao_RS": "sum"}).reset_index()
df_analise_idade = df_analise_idade.sort_values(by="Cat_Idade", key=lambda x: x.map({"Menos de 50": 0, "Entre 50 e 100": 1, "Mais de 100": 2}))

figura_3 = px.bar(df_analise_idade, x='Cat_Idade', y='Variacao_RS', text='Variacao_RS',
                  title="Variação Reais por Categoria de Idade", color="Cat_Idade", height=500, width=800)

figura_3.update_yaxes(tickprefix="R$ ")
figura_3.update_traces(texttemplate="R$ %{text:,.2f}")
figura_3.add_hline(y=0, line_color='black', line_dash="dot")

figura_3.show()


Pro fim, foi demonstrado que uma análise de dados, apesar de menos visual a princípio, é mais eficiente e profunda ao se utilizar ferramentas em Python, suas bibliotecas permitem manipulações cada vez mais detalhadas e com extração de informações interessantes para as tomadas de decisão.

Lembrando que essa é uma exploração dos dados, que por não se ter uma pergunta bem definida não extrai o máximo desse banco de dados, e o próprio BD perde seu valor ao longo tempo, já que é um recorte de um dia na bolsa, pode ter maior valor agregado se anaálisado junto a uma série temporal maior, ou com mais informações do negócio a ser tratado.

Nos vemos na próxima aula.!